# 0.1. imports

In [1]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import ecospold2matrix as e2m
import pymrio
#import brightway2 as bw
# type hints
from ecospold2matrix import ecospold2matrix
from pymrio import IOSystem
# data science
import pandas as pd
# deep copy
import copy

# 0.2. config

In [2]:
with open('../config.yaml', 'r') as filestream:
    config = yaml.load(filestream, Loader = yaml.FullLoader)

# 0.3. file paths

## 0.3.1. directories

In [3]:
%%capture
print(path_dir_data := Path(Path.home(), config['path_dir_data']))
print(path_dir_data_raw := Path(path_dir_data, config['path_dir_data_raw']))
# exiobase
print(path_dir_exiobase := Path(path_dir_data_raw, 'IOT_2011_pxp'))

## 0.3.2. files

In [16]:
%%capture
print(path_file_A := Path(path_dir_exiobase, 'A.txt'))
print(path_file_Y := Path(path_dir_exiobase, 'Y.txt'))
print(path_file_x := Path(path_dir_exiobase, 'x.txt'))
print(path_file_Z := Path(path_dir_exiobase, 'Z.txt'))

# 1. load files

In [5]:
def read_csv(filepath: Path) -> pd.DataFrame:
    df = pd.read_csv(
        filepath_or_buffer = filepath,
        sep = ',',
        engine = 'python',
        encoding = 'utf-8',
        index_col = None
    )
    return df

In [6]:
def read_ecoinvent_txt(filepath: Path) -> pd.DataFrame:
    df = pd.read_csv(
        filepath_or_buffer = filepath,
        sep = '\t',
        engine = 'python',
        encoding = 'utf-8',
        index_col = None
    )
    return df

In [17]:
A = read_ecoinvent_txt(path_file_A)
Y = read_ecoinvent_txt(path_file_Y)
x = read_ecoinvent_txt(path_file_x)
Z = read_ecoinvent_txt(path_file_Z)

# 2. investigation

#### 2.1. `A` ... A-matrix / inter-industry coefficients matrix / direct requirements matrix / technical coefficients matrix

In [8]:
A

,region,Unnamed: 1,AT,AT.1,AT.2,AT.3,AT.4,AT.5,AT.6,AT.7,...,WM.190,WM.191,WM.192,WM.193,WM.194,WM.195,WM.196,WM.197,WM.198,WM.199
0,sector,NaN,Paddy rice,Wheat,Cereal grains nec,"Vegetables, fruit, nuts",Oil seeds,"Sugar cane, sugar beet",Plant-based fibers,Crops nec,...,Paper for treatment: landfill,Plastic waste for treatment: landfill,Inert/metal/hazardous waste for treatment: lan...,Textiles waste for treatment: landfill,Wood waste for treatment: landfill,Membership organisation services n.e.c. (91),"Recreational, cultural and sporting services (92)",Other services (93),Private households with employed persons (95),Extra-territorial organizations and bodies
1,region,sector,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AT,Paddy rice,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AT,Wheat,0,0.0275637958089,0,0,0,0,0,0.00237021532833,...,2.3991062368e-06,2.71537203877e-06,2.23303493344e-06,1.53371498677e-06,1.74376592493e-06,2.09522973993e-06,9.34332578764e-07,2.42255926378e-06,3.1369621488e-07,0
4,AT,Cereal grains nec,0,0,0.0118816694099,0,0,0,0,0.00236854095084,...,1.82341715156e-05,2.15465344705e-05,1.69766940357e-05,1.19811955708e-05,1.38300383109e-05,1.89726483175e-06,6.32165277423e-06,1.56253956915e-05,2.36343724976e-06,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9797,WM,Membership organisation services n.e.c. (91),0,1.90753403716e-09,1.69946186873e-09,1.8387509863e-09,1.00411099201e-09,1.83442430149e-09,0,2.3119385787e-09,...,6.7257672153e-05,7.86005116203e-05,6.50530142052e-05,9.65196833857e-05,9.01454368753e-05,0.00448039335298,0.00177015248762,0.000702073933096,0.000398534290946,0
9798,WM,"Recreational, cultural and sporting services (92)",0,1.78007698722e-06,1.59460434851e-06,1.72877112069e-06,9.18948001033e-07,1.67483265701e-06,0,2.15558767237e-06,...,0.000226483732059,0.000268746429196,0.000272035648825,0.000287375699511,0.000290660307233,0.00695326280715,0.0615556966918,0.00504811427749,3.95621025918e-07,0
9799,WM,Other services (93),0,1.95278927867e-07,1.73948101102e-07,1.88168408819e-07,1.02693986681e-07,1.87721485124e-07,0,2.36691278366e-07,...,0.000195906361481,0.000222650626255,0.00022469501343,0.000290467508703,0.000276496767682,0.00454165936039,0.00330771340928,0.0128511898936,4.97510385678e-07,0
9800,WM,Private households with employed persons (95),0,0,0,0,0,0,0,0,...,8.23229845473e-05,7.87521508903e-05,6.7995500732e-05,9.85091362051e-05,8.7188063077e-05,0.000430606637559,0.000336531088871,0.000402184472766,0.00244440155536,0


#### 2.2. `Y` ... final demand

In [9]:
Y

,region,Unnamed: 1,AT,AT.1,AT.2,AT.3,AT.4,AT.5,AT.6,BE,...,WF.4,WF.5,WF.6,WM,WM.1,WM.2,WM.3,WM.4,WM.5,WM.6
0,category,NaN,Final consumption expenditure by households,Final consumption expenditure by non-profit or...,Final consumption expenditure by government,Gross fixed capital formation,Changes in inventories,Changes in valuables,Exports: Total (fob),Final consumption expenditure by households,...,Changes in inventories,Changes in valuables,Exports: Total (fob),Final consumption expenditure by households,Final consumption expenditure by non-profit or...,Final consumption expenditure by government,Gross fixed capital formation,Changes in inventories,Changes in valuables,Exports: Total (fob)
1,region,sector,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AT,Paddy rice,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AT,Wheat,13.068982139,0,0,0,3.15090237451,0,0,0.0149767877671,...,0,0,0,0.297723721819,9.656051028e-07,0.000754285832617,0.00614615678803,0,0,0
4,AT,Cereal grains nec,144.681455122,0,0,0,7.51631267804,0,0,0.00273670829066,...,0,0,0,1.96751346463,3.81672070747e-06,7.68393052225e-10,0.00405339440967,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9797,WM,Membership organisation services n.e.c. (91),0.00477060988481,0.679637073107,0.0457703234395,0,0,0,0,0.0443287051964,...,0,0,0,2237.07408324,6478.0104482,1413.08504877,0,0,0,0
9798,WM,"Recreational, cultural and sporting services (92)",0,0,0,2.97528071436,0,0,0,0,...,0,0,0,12774.9685564,3696.09503174,6181.59381644,1057.84065993,0,0,0
9799,WM,Other services (93),1.65133712969,0,0.0971785467569,0,0,0,0,4.13188962194,...,0,0,0,14092.5895439,1328.36089315,214.87602711,617.230076802,0,0,0
9800,WM,Private households with employed persons (95),0,0,0,0,0,0,0,0,...,0,0,0,1104.56642655,0.176417035785,27.986090955,13.5428049176,0,0,0


#### 2.3 `x` ... gross total output

In [15]:
x

,region,sector,indout
0,AT,Paddy rice,0.000000
1,AT,Wheat,337.704974
2,AT,Cereal grains nec,786.572960
3,AT,"Vegetables, fruit, nuts",1457.871318
4,AT,Oil seeds,156.381612
...,...,...,...
9795,WM,Membership organisation services n.e.c. (91),11713.041066
9796,WM,"Recreational, cultural and sporting services (92)",30690.982874
9797,WM,Other services (93),19628.147761
9798,WM,Private households with employed persons (95),4595.225678


#### 2.4. `Z`...flow/transaction matrix

In [18]:
Z

,region,Unnamed: 1,AT,AT.1,AT.2,AT.3,AT.4,AT.5,AT.6,AT.7,...,WM.190,WM.191,WM.192,WM.193,WM.194,WM.195,WM.196,WM.197,WM.198,WM.199
0,sector,NaN,Paddy rice,Wheat,Cereal grains nec,"Vegetables, fruit, nuts",Oil seeds,"Sugar cane, sugar beet",Plant-based fibers,Crops nec,...,Paper for treatment: landfill,Plastic waste for treatment: landfill,Inert/metal/hazardous waste for treatment: lan...,Textiles waste for treatment: landfill,Wood waste for treatment: landfill,Membership organisation services n.e.c. (91),"Recreational, cultural and sporting services (92)",Other services (93),Private households with employed persons (95),Extra-territorial organizations and bodies
1,region,sector,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AT,Paddy rice,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AT,Wheat,0,9.3084309593,0,0,0,0,0,0.439935977604,...,0.00303269295789,0.00199731504594,0.00292109793464,0.000474128734933,0.000694956288677,0.0245415119859,0.0286755851737,0.0475503511899,0.00144150490185,0
4,AT,Cereal grains nec,0,0,9.34579987394,0,0,0,0,0.439625196179,...,0.0230496852119,0.0158487370686,0.0222077071622,0.00370383620683,0.00551179029215,0.0222227408868,0.194017737031,0.306697575462,0.0108605275395,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9797,WM,Membership organisation services n.e.c. (91),0,6.44183733276e-07,1.33675075198e-06,2.68066232473e-06,1.57024495141e-07,1.82811081507e-07,0,4.29119222471e-07,...,0.0850199401649,0.0578152762261,0.0850977396687,0.0298378484754,0.0359263461663,52.4790313341,54.3277196823,13.7804108982,1.8313550075,0
9798,WM,"Recreational, cultural and sporting services (92)",0,0.000601140853483,0.00125427266195,0.00252032583306,0.000143706569365,0.000166906843266,0,0.000400098910264,...,0.286296458553,0.197678726493,0.355857743214,0.0888385899823,0.115839061592,81.4438528008,1889.20483298,99.0851329545,0.00181796789723,0
9799,WM,Other services (93),0,6.59466653455e-05,0.000136822872718,0.000274325326259,1.60594511373e-05,1.8707540938e-05,0,4.39322991856e-05,...,0.247643824085,0.163772565769,0.293930081355,0.0897943839814,0.110194358516,53.1966425947,101.516975597,252.24505414,0.00228617249956,0
9800,WM,Private households with employed persons (95),0,0,0,0,0,0,0,0,...,0.104063893328,0.0579268157836,0.0889468918639,0.0304528628403,0.034747721507,5.0437132289,10.3284698852,7.89413625875,11.2325767957,0
